In [1]:
# Install required libraries
!pip install transformers datasets scikit-learn torch --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.6 MB/s eta 0:00:00


In [6]:
import pandas as pd

# Load the emotion labels and main dataset
df = pd.read_csv('/content/go_emotions_dataset.csv')
labels = pd.read_csv('/content/go_emotions_dataset.csv')

print("Sample Data:")
print(df.head())

print("\nLabel Definitions:")
print(labels)



Sample Data:
        id                                               text  \
0  eew5j0j                                    That game hurt.   
1  eemcysk   >sexuality shouldn’t be a grouping category I...   
2  ed2mah1     You do right, if you don't care then fuck 'em!   
3  eeibobj                                 Man I love reddit.   
4  eda6yn6  [NAME] was nowhere near them, he was by the Fa...   

   example_very_unclear  admiration  amusement  anger  annoyance  approval  \
0                 False           0          0      0          0         0   
1                  True           0          0      0          0         0   
2                 False           0          0      0          0         0   
3                 False           0          0      0          0         0   
4                 False           0          0      0          0         0   

   caring  confusion  ...  love  nervousness  optimism  pride  realization  \
0       0          0  ...     0            0     

In [7]:
import re

def clean_text(text):
    text = text.lower()                             # Lowercase
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)  # Remove URLs
    text = re.sub(r"@\w+", "", text)                # Remove @mentions
    text = re.sub(r"#\w+", "", text)                # Remove hashtags
    text = re.sub(r"[^a-zA-Z\s]", "", text)         # Remove special chars
    text = re.sub(r"\s+", " ", text).strip()        # Normalize whitespace
    return text

# Apply to dataset
df['clean_text'] = df['text'].apply(clean_text)

# Preview result
df[['text', 'clean_text']].head()


,text,clean_text
0,That game hurt.,that game hurt
1,>sexuality shouldn’t be a grouping category I...,sexuality shouldnt be a grouping category it m...
2,"You do right, if you don't care then fuck 'em!",you do right if you dont care then fuck em
3,Man I love reddit.,man i love reddit
4,"[NAME] was nowhere near them, he was by the Fa...",name was nowhere near them he was by the falcon


In [19]:
import pandas as pd
import numpy as np
import torch

# Load dataset
df = pd.read_csv("/content/go_emotions_dataset.csv")

# Define label columns (all columns starting from 'admiration' to 'neutral')
label_columns = df.columns[3:]  # Skipping id, text, example_very_unclear

# Convert label columns to numpy array
labels_array = df[label_columns].values.astype(int)

# Calculate total number of samples and label-wise counts
total_samples = labels_array.shape[0]
label_counts = labels_array.sum(axis=0)  # Total occurrences per label

# Compute class weights: inverse frequency
class_weights = total_samples / (label_counts + 1e-6)  # Add small value to avoid division by zero

# Normalize to mean=1
class_weights = class_weights / class_weights.mean()

# Print class weights for each emotion
for label, weight in zip(label_columns, class_weights):
    print(f"{label}: Weight = {weight:.3f}")



admiration: Weight = 0.225
amusement: Weight = 0.417
anger: Weight = 0.477
annoyance: Weight = 0.283
approval: Weight = 0.219
caring: Weight = 0.643
confusion: Weight = 0.524
curiosity: Weight = 0.398
desire: Weight = 1.010
disappointment: Weight = 0.455
disapproval: Weight = 0.337
disgust: Weight = 0.727
embarrassment: Weight = 1.557
excitement: Weight = 0.685
fear: Weight = 1.206
gratitude: Weight = 0.332
grief: Weight = 5.727
joy: Weight = 0.483
love: Weight = 0.471
nervousness: Weight = 2.130
optimism: Weight = 0.442
pride: Weight = 2.960
realization: Weight = 0.439
relief: Weight = 2.990
remorse: Weight = 1.527
sadness: Weight = 0.570
surprise: Weight = 0.699
neutral: Weight = 0.070


In [23]:
class_weights_tensor = torch.tensor(class_weights).float()
print(class_weights_tensor)

tensor([0.2250, 0.4169, 0.4768, 0.2830, 0.2188, 0.6425, 0.5238, 0.3977, 1.0098,
        0.4551, 0.3374, 0.7271, 1.5567, 0.6847, 1.2056, 0.3316, 5.7272, 0.4828,
        0.4706, 2.1295, 0.4423, 2.9604, 0.4388, 2.9902, 1.5265, 0.5704, 0.6990,
        0.0697])


In [24]:
from sklearn.model_selection import train_test_split

# Define label columns (from 'admiration' to 'neutral')
label_columns = df.columns[3:]  # skip id, text, example_very_unclear

# Split into train and temp (80/20)
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df[label_columns].values.argmax(axis=1))

# Split temp into validation and test (50/50 of the 20%)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df[label_columns].values.argmax(axis=1))

# Confirm sizes
print(f"Train size: {len(train_df)}")
print(f"Validation size: {len(val_df)}")
print(f"Test size: {len(test_df)}")


Train size: 168980
Validation size: 21122
Test size: 21123


In [26]:
df.head(5)

,id,text,example_very_unclear,admiration,amusement,anger,annoyance,approval,caring,confusion,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,eew5j0j,That game hurt.,False,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,eemcysk,>sexuality shouldn’t be a grouping category I...,True,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ed2mah1,"You do right, if you don't care then fuck 'em!",False,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,eeibobj,Man I love reddit.,False,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,eda6yn6,"[NAME] was nowhere near them, he was by the Fa...",False,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


okenization with **BERT**

In [27]:
from transformers import BertTokenizer

# Load pre-trained tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Define max token length
MAX_LEN = 128

# Tokenization function
def tokenize_batch(text_list):
    return tokenizer(
        text_list,
        padding='max_length',
        truncation=True,
        max_length=MAX_LEN,
        return_tensors='pt'
    )


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [28]:
import torch
from torch.utils.data import Dataset

# Define custom dataset class
class GoEmotionsDataset(Dataset):
    def __init__(self, texts, labels):
        self.encodings = tokenize_batch(texts)
        self.labels = torch.tensor(labels, dtype=torch.float32)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = self.labels[idx]
        return item


In [29]:
# Prepare datasets
train_dataset = GoEmotionsDataset(train_df['text'].tolist(), train_df[label_columns].values)
val_dataset = GoEmotionsDataset(val_df['text'].tolist(), val_df[label_columns].values)
test_dataset = GoEmotionsDataset(test_df['text'].tolist(), test_df[label_columns].values)


In [30]:
from transformers import BertForSequenceClassification

# Load BERT model for multi-label classification
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=len(label_columns),
    problem_type="multi_label_classification"
)


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [32]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch", # Changed from evaluation_strategy to eval_strategy
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="f1"
)

In [38]:
def compute_metrics(pred):
    logits, labels = pred
    probs = 1 / (1 + np.exp(-logits))  # sigmoid
    preds = (probs > 0.5).astype(int)

    # Prevent failure if there are no positives
    try:
        f1_micro = f1_score(labels, preds, average='micro', zero_division=0)
        f1_macro = f1_score(labels, preds, average='macro', zero_division=0)
        h_loss = hamming_loss(labels, preds)
    except ValueError as e:
        print("Metric computation error:", e)
        return {}

    return {
        'f1_micro': f1_micro,
        'f1_macro': f1_macro,
        'hamming_loss': h_loss
    }


In [39]:
num_labels = len(label_columns)  # Should be 28
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=num_labels,
    problem_type="multi_label_classification"
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [44]:
from sklearn.metrics import f1_score, hamming_loss
import torch
import numpy as np

model.eval()  # Set model to evaluation mode

all_labels = []
all_preds = []

with torch.no_grad():
    for batch in torch.utils.data.DataLoader(test_dataset, batch_size=32):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        # Apply sigmoid to get probabilities
        probs = torch.sigmoid(logits)

        # Convert probs to binary predictions with threshold 0.5
        preds = (probs > 0.5).int()

        all_labels.append(labels.cpu().numpy())
        all_preds.append(preds.cpu().numpy())

# Concatenate all batches
all_labels = np.vstack(all_labels)
all_preds = np.vstack(all_preds)

# Calculate metrics
f1_micro = f1_score(all_labels, all_preds, average='micro')
f1_macro = f1_score(all_labels, all_preds, average='macro')
h_loss = hamming_loss(all_labels, all_preds)

print(f"F1 Micro: {f1_micro:.4f}")
print(f"F1 Macro: {f1_macro:.4f}")
print(f"Hamming Loss: {h_loss:.4f}")


KeyboardInterrupt: 

In [45]:
from sklearn.metrics import f1_score, hamming_loss
import torch
import numpy as np

model.eval()  # Set model to evaluation mode

all_labels = []
all_preds = []

with torch.no_grad():
    for batch in torch.utils.data.DataLoader(test_dataset, batch_size=32):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        # Apply sigmoid to get probabilities
        probs = torch.sigmoid(logits)

        # Convert probs to binary predictions with threshold 0.5
        preds = (probs > 0.5).int()

        all_labels.append(labels.cpu().numpy())
        all_preds.append(preds.cpu().numpy())

# Concatenate all batches
all_labels = np.vstack(all_labels)
all_preds = np.vstack(all_preds)

# Calculate metrics
f1_micro = f1_score(all_labels, all_preds, average='micro')
f1_macro = f1_score(all_labels, all_preds, average='macro')
h_loss = hamming_loss(all_labels, all_preds)

print(f"F1 Micro: {f1_micro:.4f}")
print(f"F1 Macro: {f1_macro:.4f}")
print(f"Hamming Loss: {h_loss:.4f}")


KeyboardInterrupt: 

In [46]:
def predict_emotions(texts):
    model.eval()

    # Tokenize texts
    inputs = tokenizer(texts, padding=True, truncation=True, max_length=128, return_tensors="pt")

    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.sigmoid(outputs.logits)
        preds = (probs > 0.5).int()

    return preds.cpu().numpy()

# Example real-world texts
samples = [
    "I love the new update, it's fantastic!",
    "This product is terrible and frustrating.",
    "I'm not sure if I like it or not.",
]

predictions = predict_emotions(samples)

for text, pred in zip(samples, predictions):
    emotion_indices = np.where(pred == 1)[0]
    emotion_labels = [label_columns[i] for i in emotion_indices]
    print(f"Text: {text}\nPredicted emotions: {emotion_labels}\n")


Text: I love the new update, it's fantastic!
Predicted emotions: ['admiration', 'amusement', 'annoyance', 'confusion', 'curiosity', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'optimism', 'neutral']

Text: This product is terrible and frustrating.
Predicted emotions: ['admiration', 'amusement', 'annoyance', 'confusion', 'curiosity', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'optimism', 'neutral']

Text: I'm not sure if I like it or not.
Predicted emotions: ['admiration', 'amusement', 'annoyance', 'approval', 'confusion', 'curiosity', 'disgust', 'embarrassment', 'fear', 'gratitude', 'joy', 'optimism', 'surprise', 'neutral']

